In [1]:

import re
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os

MODEL_PRETRAIN = 'indobenchmark/indobert-base-p2'
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(MODEL_PRETRAIN,  truncation=True)


In [2]:
MAPING_CUT_THRESH = 15

In [3]:
!pip3 install -q sklearn-crfsuite

In [4]:
import pickle

In [5]:
import pandas as pd

In [6]:
train = pd.read_parquet('C:/Users/Ale/Desktop/jupyter_folder/FYP/CRF & PQT files/train.pqt')
val = pd.read_parquet('C:/Users/Ale/Desktop/jupyter_folder/FYP/CRF & PQT files/val.pqt')
test = pd.read_parquet('C:/Users/Ale/Desktop/jupyter_folder/FYP/CRF & PQT files/test.pqt')

In [7]:
import pickle

In [8]:
with open('C:/Users/Ale/Desktop/jupyter_folder/FYP/CRF & PQT files/train_sent.pkl', 'rb') as f:
    train_sent = pickle.load(f)

In [9]:
with open('C:/Users/Ale/Desktop/jupyter_folder/FYP/CRF & PQT files/val_sent.pkl', 'rb') as f:
    val_sent = pickle.load(f)

In [10]:
with open('C:/Users/Ale/Desktop/jupyter_folder/FYP/CRF & PQT files/test_sent.pkl', 'rb') as f:
    test_sent = pickle.load(f)

In [11]:
with open('C:/Users/Ale/Desktop/jupyter_folder/FYP/CRF & PQT files/crf.pkl', 'rb') as f:
    crf = pickle.load(f)

In [12]:
with open('C:/Users/Ale/Desktop/jupyter_folder/FYP/CRF2/maping_poi.pkl', 'rb') as f:
    maping_poi = pickle.load(f)
    
with open('C:/Users/Ale/Desktop/jupyter_folder/FYP/CRF2/maping_street.pkl', 'rb') as f:
    maping_street = pickle.load(f)

In [13]:
val_marginals =crf.predict_marginals(val_sent)

In [14]:
val['marginals'] = val_marginals

In [15]:
train['marginals'] =crf.predict_marginals(train_sent)

In [16]:
test['marginals'] =crf.predict_marginals(test_sent)

In [17]:
def naive_tokenize(x):
    if type(x) == str:
        return re.split('(\W)', x)
    return x


def insert_space(ref, tokens):
  match = ''.join(tokens)

  p = 0
  while p < len(ref):

    m1 = p
    m2 = 0
    while m1 < len(ref) and m2 < len(match):
      if ref[m1] == ' ':
        m1 += 1
        continue

      if ref[m1] != match[m2]:
        break

      m1 += 1
      m2 += 1
    
    if m2 == len(match):
      return ref[p:m1].strip()
    
    p += 1
  
  return ''


def get_maping(maping, tokens, idx, tresh=0):
    token = tokens[idx]
    if token not in maping:
        return token
    try:
        max_token = max(maping[token], key=maping[token].get)
        if maping[token][max_token] > tresh:
            return max_token
        else:
            return token
    except:
        return token

In [18]:
import re
def is_roman_numeral(txt): 
    return bool(
        re.search(
            r"^M{0,3}(CM|CD|D?C{0,3})(XC|XL|L?X{0,3})(IX|IV|V?I{0,3})$",\
            txt.upper()
        )
    )

In [19]:
def convert_token(token):
    return token
#     if is_roman_numeral(token):
#         return "<ROMAN_NUMERAL>"
#     elif token.isdigit():
#         return "<DIGIT>"
#     else:
#         return token
#     elif token.isalpha():
#         return token
#     else:
#         _result = set()
#         for char in token:
#             if char.isalpha():
#                 _result.add('ALPHA')
#             elif char.isdigit():
#                 _result.add('DIGIT')
#         return "<" + "_".join(sorted(list(_result))) + ">"


def generate_n_gram(_list, n=2):
    if len(_list) < n:
        return []
    else:
        return [[_list[i+p] for p in range(n)] for i in range(len(_list)-(n-1))]

In [20]:
def bigram_maping(tokens, poi_tokens, target_bigram):
    pair_token = generate_n_gram(tokens)
    pair_poi = generate_n_gram(poi_tokens)
    for poi1, poi2 in pair_poi:
        for token1, token2 in pair_token:
#             token1_convert = convert_token(token1)
#             poi1_convert =  convert_token(poi1)
            
            token2_convert = convert_token(token2)
            poi2_convert =  convert_token(poi2)

            if poi1.startswith(token1) and poi2.startswith(token2):
                target_bigram[(token1, token2_convert)][(poi1,poi2)] += 1

In [21]:
def trigram_maping(tokens, poi_tokens, target_trigram):
    thrice_token = generate_n_gram(tokens, 3)
    thrice_poi = generate_n_gram(poi_tokens, 3)
    for poi1, poi2, poi3 in thrice_poi:
        for token1, token2, token3 in thrice_token:
            if poi1.startswith(token1) and poi2.startswith(token2) and poi3.startswith(token3):
                target_trigram[(token1, token2, token3)][(poi1,poi2, poi3)] += 1

In [22]:
def get_maping2(maping, token, tresh):
    if token not in maping:
        return token
    try:
        max_token = max(maping[token], key=maping[token].get)
        if max_token in ['<>', '<DIGIT>', '<ROMAN_NUMERAL>']:
            return token
        if maping[token][max_token] > tresh:
            return max_token
        else:
            return token
    except:
        return token

In [23]:
def reformat_target(addr, bigram_target, trigram_target, tresh=0):
    if len(addr) > 2:
        for idx in range(len(addr) -2):
            a, b, c = addr[idx], addr[idx+1], addr[idx+2]
            a, b, c = get_maping2(trigram_target, (a, b, c), tresh-1)
            addr[idx] = a
            addr[idx+1] = b
            addr[idx+2] = c

    if len(addr) > 1:
        for idx in range(len(addr) -1):
            a, b = get_maping2(bigram_target, (addr[idx], convert_token(addr[idx+1])), tresh)
            addr[idx] = a
            addr[idx+1] = b
    return addr

In [24]:
OTHER = 'O'
def generate_prediction1(predicted, raw_address_tokens, raw_address, use_maping=False, maping_poi=None, maping_street=None, debug=False, reformat_target=None):
    _poi = []
    _street = []
    
    for idx, token in enumerate(raw_address_tokens):
        if predicted[idx] != OTHER:
            tag = predicted[idx].split('-')[1]
            _token = token
            if tag == 'POI':
                _poi.append(_token)
            else:
                _street.append(_token)
                
    if reformat_target is not None:
        _poi = reformat_target['poi'](_poi)
        _street = reformat_target['street'](_street)

    _poi = tokenizer.convert_tokens_to_string(_poi)
    _street = tokenizer.convert_tokens_to_string(_street)
    
    if debug:
        print(_poi)
        print(_street)
        
    if use_maping:
        
        _poi_naive_tokenized = naive_tokenize(_poi)
        _street_naive_tokenized = naive_tokenize(_street)
        if debug:
            print(_poi_naive_tokenized)
            print(_street_naive_tokenized)
        
        _poi = [get_maping(maping_poi, _poi_naive_tokenized, idx) for idx, token in enumerate(_poi_naive_tokenized)]
#         _poi = [get_maping(maping_street, _poi, idx) for idx, token in enumerate(_poi)]
        
        _street = [get_maping(maping_street, _street_naive_tokenized, idx) for idx, token in enumerate(_street_naive_tokenized)]
#         _street = [get_maping(maping_poi, _street, idx) for idx, token in enumerate(_street)]
        
        if debug:
            print('-'*20)
            print(_poi)
            print(_street)
        
        new_poi = insert_space(raw_address, [x for x in _poi if x != ' '])
        new_street = insert_space(raw_address, [x for x in _street if x != ' '])

        if debug:
            print(new_poi)
            print(new_street)
        
        if len(new_street) < 1:
            new_street = ''.join(_street)
        if len(new_poi) < 1:
            new_poi = ''.join(_poi)
        
    return  new_poi + '/' + new_street

In [25]:
from collections import defaultdict

In [26]:
from tqdm import tqdm
tqdm.pandas()

In [27]:

bigram_poi = defaultdict(lambda: defaultdict(lambda :0))
_ = train[['clean_tokens', 'POI']].progress_apply(lambda x: bigram_maping(*x, bigram_poi), axis=1)

100%|██████████| 224740/224740 [00:06<00:00, 32665.65it/s]


In [28]:

bigram_street = defaultdict(lambda: defaultdict(lambda :0))
_ = train[['clean_tokens', 'street']].progress_apply(lambda x: bigram_maping(*x, bigram_street), axis=1)

100%|██████████| 224740/224740 [00:07<00:00, 28784.07it/s]


In [29]:

trigram_poi = defaultdict(lambda: defaultdict(lambda :0))
_ = train[['clean_tokens', 'POI']].progress_apply(lambda x: trigram_maping(*x, trigram_poi), axis=1)

100%|██████████| 224740/224740 [00:05<00:00, 42393.95it/s]


In [30]:

trigram_street = defaultdict(lambda: defaultdict(lambda :0))
_ = train[['clean_tokens', 'street']].progress_apply(lambda x: trigram_maping(*x, trigram_street), axis=1)

100%|██████████| 224740/224740 [00:09<00:00, 24855.50it/s]


In [31]:
val.head(2)

raw_address        POI/street  \
id                                                             
99792     kemu 44 ketabang rt 9 10 genteng             /kemu   
24401  med utara xxv b medokan ayu rungkut  /med utara xxv b   

                                      clean_tokens POI                street  \
id                                                                             
99792     [kemu, 44, ketabang, rt, 9, 10, genteng]  []                [kemu]   
24401  [med, utara, xxv, b, medokan, ayu, rungkut]  []  [med, utara, xxv, b]   

                                                  labels  \
id                                                         
99792                       [B-STREET, O, O, O, O, O, O]   
24401  [B-STREET, I-STREET, I-STREET, I-STREET, O, O, O]   

                                            tokens          predict1  \
id                                                                     
99792     [kemu, 44, ketabang, rt, 9, 10, genteng]             /kemu   
24401  [med, utara, xxv, b, medokan, ayu, rungkut]  /med utara xxv b   

                                               pred_list  \
id                                                         
99792                       [B-STREET, O, O, O, O, O, O]   
24401  [B-STREET, I-STREET, I-STREET, I-STREET, O, O, O]   

                                                  y_list  \
id                                                         
99792                       [B-STREET, O, O, O, O, O, O]   
24401  [B-STREET, I-STREET, I-STREET, I-STREET, O, O, O]   

                                               marginals  
id                                                        
99792  [{'O': 0.06676467256726396, 'B-POI': 0.0007979...  
24401  [{'O': 0.06165722614154339, 'B-POI': 1.2122792...

In [32]:
from tqdm import tqdm
train['predict3'] = [
    generate_prediction1(
        train['pred_list'].iloc[i],
        train['tokens'].iloc[i],
        train['raw_address'].iloc[i],
        use_maping=True,
        maping_poi=maping_poi,
        maping_street=maping_street
    )
    for i in tqdm(range(train.shape[0]), total=train.shape[0])
]

100%|██████████| 224740/224740 [00:26<00:00, 8422.02it/s] 


In [33]:
(train['predict3'] == train['POI/street']).mean()

0.7766797187861529

In [34]:
from tqdm import tqdm
train['predict4'] = [
    generate_prediction1(
        train['pred_list'].iloc[i],
        train['tokens'].iloc[i],
        train['raw_address'].iloc[i],
        use_maping=True,
        maping_poi=maping_poi,
        maping_street=maping_street,
        reformat_target = {
            'poi': lambda x: reformat_target(x, bigram_poi, trigram_poi, 3),
            'street': lambda x: reformat_target(x, bigram_street, trigram_street, 3)            
        }
    )
    for i in tqdm(range(train.shape[0]), total=train.shape[0])
]

100%|██████████| 224740/224740 [00:27<00:00, 8174.23it/s]


In [35]:
(train['predict4'] == train['POI/street']).mean()

0.780079202634155

In [37]:
from tqdm import tqdm
val['predict3'] = [
    generate_prediction1(
        val['pred_list'].iloc[i],
        val['tokens'].iloc[i],
        val['raw_address'].iloc[i],
        use_maping=True,
        maping_poi=maping_poi,
        maping_street=maping_street
    )
    for i in tqdm(range(val.shape[0]), total=val.shape[0])
]

100%|██████████| 74914/74914 [00:07<00:00, 10492.97it/s]


In [38]:
(val['predict3'] == val['POI/street']).mean()

0.6535360546760285

In [40]:
from tqdm import tqdm
val['predict4'] = [
    generate_prediction1(
        val['pred_list'].iloc[i],
        val['tokens'].iloc[i],
        val['raw_address'].iloc[i],
        use_maping=True,
        maping_poi=maping_poi,
        maping_street=maping_street,
        reformat_target = {
            'poi': lambda x: reformat_target(x, bigram_poi, trigram_poi, 2),
            'street': lambda x: reformat_target(x, bigram_street, trigram_street, 2)
        }
    )
    for i in tqdm(range(val.shape[0]), total=val.shape[0])
]

100%|██████████| 74914/74914 [00:09<00:00, 7891.27it/s]


In [41]:
(val['predict4'] == val['POI/street']).mean()

0.6555783965613904

In [43]:
from sklearn.model_selection import train_test_split

In [44]:
def proc_sent(marginals, sent):
    for idx, tok in enumerate(sent):
        marginals[idx]['b'] = 1
        marginals[idx]['v0'] = tok['v0']        
        marginals[idx]['v1'] = tok['v1']
        marginals[idx]['v2'] = tok['v2']
        marginals[idx]['v3'] = tok['v3']
        marginals[idx]['v4'] = tok['v4']
    return marginals

In [45]:
val['sent'] = val_sent

In [46]:
val['marginals'] = val[['marginals', 'sent']].progress_apply(lambda x: proc_sent(*x), axis=1)

100%|██████████| 74914/74914 [00:02<00:00, 33889.40it/s]


In [47]:
train['sent'] = train_sent

In [48]:
train['marginals'] = train[['marginals', 'sent']].progress_apply(lambda x: proc_sent(*x), axis=1)

100%|██████████| 224740/224740 [00:08<00:00, 26410.43it/s]


In [49]:
test['sent'] = test_sent

In [50]:
test['marginals'] = test[['marginals', 'sent']].progress_apply(lambda x: proc_sent(*x), axis=1)

100%|██████████| 50000/50000 [00:01<00:00, 27748.49it/s]


In [51]:
SEED = 2021
train_val, val_val = train_test_split(val, shuffle=True, random_state=SEED, test_size=0.3)

In [52]:
from sklearn_crfsuite import CRF
ncrf = CRF(algorithm='lbfgs',
          c1=0.0,
          c2=0.0,
          max_iterations=300,
          all_possible_transitions=False,
         verbose=True)

In [53]:
try:
    ncrf.fit(
        train_val['marginals'].append(train['marginals']), train_val['labels'], val_val['marginals'], val_val['labels']
    )
except AttributeError:
    pass

loading dev data to CRFsuite:   7%|▋         | 1660/22475 [00:00<00:01, 16596.14it/s]

loading dev data to CRFsuite: 100%|██████████| 22475/22475 [00:01<00:00, 16675.40it/s]



Holdout group: 2

Feature generation
type: CRF1d
feature.minfreq: 0.000000
feature.possible_states: 0
feature.possible_transitions: 0
0....1....2....3....4....5....6....7....8....9....10
Number of features: 70
Seconds required: 0.155

L-BFGS optimization
c1: 0.000000
c2: 0.000000
num_memories: 6
max_iterations: 300
epsilon: 0.000010
stop: 10
delta: 0.000010
linesearch: MoreThuente
linesearch.max_iterations: 20

Iter 1   time=0.90  loss=383915.86 active=70    precision=0.124  recall=0.200  F1=0.153  Acc(item/seq)=0.620 0.110  feature_norm=1.00
Iter 2   time=0.56  loss=299048.06 active=70    precision=0.124  recall=0.200  F1=0.153  Acc(item/seq)=0.620 0.110  feature_norm=1.71
Iter 3   time=0.57  loss=205138.55 active=70    precision=0.842  recall=0.505  F1=0.534  Acc(item/seq)=0.778 0.369  feature_norm=2.68
Iter 4   time=0.44  loss=112824.13 active=70    precision=0.875  recall=0.878  F1=0.876  Acc(item/seq)=0.907 0.734  feature_norm=5.05
Iter 5   time=0.40  loss=103005.55 active=70    

In [54]:
(val_val['predict4'] == val_val['POI/street']).mean()

0.6532146829810901

In [55]:
val_val['new_pred_list'] = ncrf.predict(val_val['marginals'])

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [56]:
from tqdm import tqdm
val_val['predict5'] = [
    generate_prediction1(
        val_val['new_pred_list'].iloc[i],
        val_val['tokens'].iloc[i],
        val_val['raw_address'].iloc[i],
        use_maping=True,
        maping_poi=maping_poi,
        maping_street=maping_street,
        reformat_target = {
            'poi': lambda x: reformat_target(x, bigram_poi, trigram_poi, 2),
            'street': lambda x: reformat_target(x, bigram_street, trigram_street, 2)
        }
    )
    for i in tqdm(range(val_val.shape[0]), total=val_val.shape[0])
]

100%|██████████| 22475/22475 [00:02<00:00, 8672.47it/s]
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app


In [57]:
(val_val['predict5'] == val_val['POI/street']).mean()

0.6610456062291435

In [58]:
(val_val['predict5'] == val_val['POI/street']).mean()

0.6610456062291435

In [59]:
train['new_pred_list'] = ncrf.predict(train['marginals'])

In [60]:
from tqdm import tqdm
train['predict5'] = [
    generate_prediction1(
        train['new_pred_list'].iloc[i],
        train['tokens'].iloc[i],
        train['raw_address'].iloc[i],
        use_maping=True,
        maping_poi=maping_poi,
        maping_street=maping_street,
        reformat_target = {
            'poi': lambda x: reformat_target(x, bigram_poi, trigram_poi, 2),
            'street': lambda x: reformat_target(x, bigram_street, trigram_street, 2)
        }
    )
    for i in tqdm(range(train.shape[0]), total=train.shape[0])
]

100%|██████████| 224740/224740 [00:27<00:00, 8202.81it/s]


In [61]:
(train['predict5'] == train['POI/street']).mean()

0.7194090949541693

In [62]:
val['new_pred_list'] = ncrf.predict(val['marginals'])
val['predict5'] = [
    generate_prediction1(
        val['new_pred_list'].iloc[i],
        val['tokens'].iloc[i],
        val['raw_address'].iloc[i],
        use_maping=True,
        maping_poi=maping_poi,
        maping_street=maping_street,
        reformat_target = {
            'poi': lambda x: reformat_target(x, bigram_poi, trigram_poi, 2),
            'street': lambda x: reformat_target(x, bigram_street, trigram_street, 2)
        }
    )
    for i in tqdm(range(val.shape[0]), total=val.shape[0])
]

100%|██████████| 74914/74914 [00:09<00:00, 7656.65it/s]


In [63]:
(val['predict5'] == val['POI/street']).mean()

0.6633873508289505

In [64]:
train['pred_lst'] = train['pred_list'].apply(lambda x: 'x'.join(x))
train['y_true_lst'] = train['y_list'].apply(lambda x: 'x'.join(x))

In [65]:
# train['predict5b'] = train['predict5'].apply(lambda x: x.replace(
#     'bimbi bela', 'bimbingan belajar').replace(
#     'sd neg', 'sd negeri').replace(
#     'smp neg', 'smp negeri').replace(
#     'sma neg', 'sma negeri').replace(
#     'al - ', 'al-')                       
# )

def smart_replace(x, key, target):
    if target not in x:
        return x.replace(key, target)
    return x

train['predict5b'] = train['predict5']
for k in bigram_poi:
    for k2 in bigram_poi[k]:
        if bigram_poi[k][k2] > MAPING_CUT_THRESH and k != k2:
            print(k, k2, bigram_poi[k][k2])
            train['predict5b'] = train['predict5b'].progress_apply(lambda x: smart_replace(x, k[0] + ' ' + k[1], k2[0] + ' ' + k2[1] ))

 20%|█▉        | 44035/224740 [00:00<00:00, 440347.27it/s]

('beng', 'las') ('bengkel', 'las') 71


 20%|██        | 45851/224740 [00:00<00:00, 458503.88it/s]

('maje', 'tak') ('majelis', 'taklim') 31


 20%|██        | 45361/224740 [00:00<00:00, 453609.35it/s]

...

 24%|██▍       | 53867/224740 [00:00<00:00, 538665.38it/s]

('bank', 'tabu') ('bank', 'tabungan') 17


 20%|██        | 45086/224740 [00:00<00:00, 450855.06it/s]

('sd', 'inp') ('sd', 'inpres') 19


100%|██████████| 224740/224740 [00:00<00:00, 484421.17it/s]


In [66]:
(train['predict5b'] == train['POI/street']).mean(), (train['predict5'] == train['POI/street']).mean()

(0.724908783483136, 0.7194090949541693)

In [67]:
(train['predict5b'] == train['POI/street']).mean(), (train['predict5'] == train['POI/street']).mean()

(0.724908783483136, 0.7194090949541693)

In [68]:
val['predict5b'] = val['predict5']
for k in bigram_poi:
    for k2 in bigram_poi[k]:
        if bigram_poi[k][k2] > MAPING_CUT_THRESH and k != k2:
#             print(k, k2, bigram_poi[k][k2])
            val['predict5b'] = val['predict5b'].progress_apply(lambda x: smart_replace(x, k[0] + ' ' + k[1], k2[0] + ' ' + k2[1] ))

100%|██████████| 74914/74914 [00:00<00:00, 471608.65it/s]


In [69]:
(val['predict5b'] == val['POI/street']).mean(), (val['predict5'] == val['POI/street']).mean()

(0.6683396961849587, 0.6633873508289505)

In [70]:
(val['predict5b'] == val['POI/street']).mean(), (val['predict5'] == val['POI/street']).mean()

(0.6683396961849587, 0.6633873508289505)

In [71]:
val['predict5c'] = val['predict5b'].apply(lambda x: x.replace('al - ', 'al-'))

In [72]:
(val['predict5c'] == val['POI/street']).mean()

0.6689003390554502

In [73]:
# val[(val['predict5c'] == val['POI/street']) & (val['predict5c'] == val['POI/street'])]

In [74]:
val['pred_lst'] = val['pred_list'].apply(lambda x: 'x'.join(x))
val['y_true_lst'] = val['y_list'].apply(lambda x: 'x'.join(x))

In [75]:
val[(val['predict5c'] != val['POI/street']) & (val['pred_lst'] == val['y_true_lst'])][['predict5c', 'POI/street', 'raw_address']].head(50)

predict5c  \
id                                                       
38004             sumberkah jaya reklame/raya kam unud   
200040                bela apartemen icon/kebon man iv   
107149          santika depart store & furniture/s par   
94174                                         /wij vii   
215730                 cent park lantai gf/letj s. par   
6898                                                 /   
50705                             adhi chandra export/   
201021        bumdes bumiputera asuransi koe/kol. sugi   
99690              omega jobel indonesia cv,/man adi i   
28410                                     /irian barat   
273912                 bintang baru bang/kh hasyim ash   
99857                                      /teuku umar   
27987              prak drg. i g a hartini/kuruk setra   
83740                   roemah macanre salsa/pasir kal   
122724                      depan kantor bup/jl. panji   
216332                     terminal busana pilangsari/   
90836                          japan tailor/sul hasanu   
249033                                /masjid at taqwa   
94851                                   /tuba indah ii   
134088  sate padang ajo bukit selatan metro/soek hatta   
228282                                         /pangan   
96000                                                /   
3275     kantor politeknik api yogyakarta/tam bayan 17   
149086                        ud bangun sari/raya pang   
226111             warung seblak wa teni/gun gun molek   
43678                                  kali baru/gg. 1   
131833                          gudang pernik/let supr   
268022                         toko burung rohman/lame   
294646        anugrah ganti oli/prabumulih - belimbing   
177867                                               /   
231024                                       /datu ked   
81079                       chin food pak sarana/sriwi   
205520   pt parabola div bali - padang baitul manggis/   
129037                  mart & pukis badoet/raya lapan   
74153                            depan grap telkomsel/   
276349                                           /mojo   
115445                                   /r.m marg djo   
44031                                      /sid gg. iv   
159359                                               /   
158724                  solaria singkawang grand mall/   
231625                            pt enka tailor jaya/   
96031                            edelweis gas/bha abri   
78756                          western union/raya kamo   
62229                                                /   
281462                                               /   
136437        rm pilihan saiyo/perum taman danau indah   
285020                           ff karaoke/leng kecil   
95511                    ozora boutique/prof. dr. buny   
45304                             sma alas al khoziny/   
243596                                   /laksa malaha   

                                               POI/street  \
id                                                          
38004                   sumber jaya reklame/raya kam unud   
200040               belakang apartemen icon/kebon man iv   
107149      santika department store & furniture/s parman   
94174                                         /wijaya vii   
215730                 central park lantai gf/letj s. par   
6898                                      pantai pelangi/   
50705                             adhitya chandra export/   
201021           bumida bumiputera asuransi koe/kol. sugi   
99690                 omega jobel indonesia. cv/man adi i   
28410                                kopitiam/irian barat   
273912                bintang baru bangunan/kh hasyim ash   
99857                                  3second/teuku umar   
27987              praktik drg. i g a hartini/kuruk setra   
83740                  roemah macaroni salsa/pasir kaliki   
122724                      depan kantor bupati/jl. 

In [76]:
maping_poi['bela']

{'belakang': 7,
 'bela': 31,
 'belasia': 1,
 'belajar': 6,
 'belanja': 1,
 'belanti': 1,
 'belalang': 1,
 'belawan': 1}

In [77]:
# train['pred_4_lst'] = train['pred_list'].apply(lambda x: 'x'.join(x))
# train['y_true_lst'] = train['y_list'].apply(lambda x: 'x'.join(x))

# train['pred_4_poi'] = train['predict4'].str.split('/', n = 2, expand = True)[0]
# train['pred_4_street'] = train['predict4'].str.split('/', n = 2, expand = True)[1]

# tmp = train[(train['predict4'] != train['POI/street']) & (train['y_true_lst'] == train['pred_4_lst'])]

# q = tmp['POI/street'].apply(lambda x: '-' in x or '.' in x or ',' in x)

# 

In [78]:
def join_dict(dict1, dict2):
    dict1 = dict(dict1)
    dict2 = dict(dict2)
    
    _join_dict = {k:dict(dict1[k]) for k in dict1}
    for k in dict2:
        if k not in _join_dict:
            _join_dict[k] = dict(dict2[k])
        else:
            for kk in dict2[k]:
                if kk not in _join_dict[k]:
                    _join_dict[k][kk] = dict2[k][kk]
                else:
                    _join_dict[k][kk] += dict2[k][kk]
    return _join_dict

In [79]:

_ = val[['clean_tokens', 'POI']].progress_apply(lambda x: bigram_maping(*x, bigram_poi), axis=1)
_ = val[['clean_tokens', 'street']].progress_apply(lambda x: bigram_maping(*x, bigram_street), axis=1)
_ = val[['clean_tokens', 'POI']].progress_apply(lambda x: trigram_maping(*x, trigram_poi), axis=1)
_ = val[['clean_tokens', 'street']].progress_apply(lambda x: trigram_maping(*x, trigram_street), axis=1)

100%|██████████| 74914/74914 [00:02<00:00, 29860.68it/s]


In [80]:
test['predict4'] = [
    generate_prediction1(
        test['pred_list'].iloc[i],
        test['tokens'].iloc[i],
        test['raw_address'].iloc[i],
        use_maping=True,
        maping_poi=maping_poi,
        maping_street=maping_street,
        reformat_target = {
            'poi': lambda x: reformat_target(x, bigram_poi, trigram_poi, 2),
            'street': lambda x: reformat_target(x, bigram_street, trigram_street, 2)
        }
    )
    for i in tqdm(range(test.shape[0]), total=test.shape[0])
]

100%|██████████| 50000/50000 [00:07<00:00, 7082.56it/s]


In [81]:
test['POI/street'] = test['predict4']

In [82]:
test.reset_index()[['id', 'POI/street']].to_csv('submission_uwu.csv', index=False)

In [83]:
test['pred_marginals'] = ncrf.predict(test['marginals'])

In [84]:
test['predict5'] = [
    generate_prediction1(
        test['pred_marginals'].iloc[i],
        test['tokens'].iloc[i],
        test['raw_address'].iloc[i],
        use_maping=True,
        maping_poi=maping_poi,
        maping_street=maping_street,
        reformat_target = {
            'poi': lambda x: reformat_target(x, bigram_poi, trigram_poi, 2),
            'street': lambda x: reformat_target(x, bigram_street, trigram_street, 2)
        }
    )
    for i in tqdm(range(test.shape[0]), total=test.shape[0])
]

100%|██████████| 50000/50000 [00:05<00:00, 8824.46it/s]


In [85]:
test['POI/street'] = test['predict5']
test.reset_index()[['id', 'POI/street']].to_csv('submission_owo_with_marginal_predict.csv', index=False)

In [86]:
test['predict5b'] = test['predict5']
for k in bigram_poi:
    for k2 in bigram_poi[k]:
        if bigram_poi[k][k2] > MAPING_CUT_THRESH and k != k2:
#             print(k, k2, bigram_poi[k][k2])
            test['predict5b'] = test['predict5b'].progress_apply(lambda x: smart_replace(x, k[0] + ' ' + k[1], k2[0] + ' ' + k2[1] ))

100%|██████████| 50000/50000 [00:00<00:00, 439452.75it/s]


In [87]:
test['predict5c'] = test['predict5b'].apply(lambda x: x.replace('al - ', 'al-'))

In [88]:
test['POI/street'] = test['predict5c']
test.reset_index()[['id', 'POI/street']].to_csv('submission_corrected_with_marginal_predict.csv', index=False)

In [89]:
!head submission_uwu.csv

id,POI/street
0,/s. par
1,/angg per
2,asma laundry/mand imog
3,ud agung rej/raya nga
4,/cut mutia
5,pempek dos dapur ala/perum gar
6,tb. maranggi/
7,pura taman beji tista/
8,/


In [90]:
!head submission_owo_with_marginal_predict.csv

id,POI/street
0,/s. par
1,/angg per
2,asma laundry/mand imog
3,ud agung rej/raya nga
4,/cut mutia
5,pempek dos dapur ala/perum gar
6,tb. maranggi/
7,pura taman beji tista/
8,/


In [91]:
test[test['predict5c'] != test['predict4']][['predict4', 'predict5', 'predict5b', 'predict5c']].head(30)

predict4  \
id                                                       
13                                                   /   
22                                                   /   
40                               ruko cikitsu/jl abu 1   
44                          komplek karya baru permai/   
47                                     rezi/pasar raya   
48                                                   /   
66                                               /perw   
90                                              /pelab   
100                         tiki/bule utara kelapa gad   
110                                                  /   
113                                sd neg 3 seunuddon/   
131                                                  /   
141                                         /air camar   
144                               /tan baru tan baru v   
168                     bank rak indonesia/raya pengas   
183                                    felisa/ploso vi   
195                                                  /   
197                                           /pramuka   
204                          kunto wijono/dr man besar   
211                    ayam gepuk pak blangkon/dr. sut   
219                                masjid al-muttaqin/   
234  paguyuban pecinta batik indonesia sekar jagad,...   
237                                  munawa/tenun suka   
250                                                  /   
259  sekolah menengah pertama neg 19 samarinda/poro...   
262                            slogi kuliner/teru kopo   
266                                                  /   
286                                 singapore station/   
294                                                  /   
295                                                  /   

                                              predict5  \
id                                                       
13                               warung bakso latansa/   
22                                            /gg. pan   
40                                 ruko cikitsu/jl abu   
44   komplek karya baru permai/jl. karya baru  jl. ...   
47                                         /pasar raya   
48                                  cv pondok raminih/   
66                                                   /   
90                koperasi simpan pinjam sinar pelita/   
100                             /bule utara kelapa gad   
110                                        /singojuruh   
113                                sd neg 3 seunuddon/   
131                       california fried chicken/dip   
141                                                  /   
144                                        /tan baru v   
168                     bank rak indonesia/raya pengas   
183                                          /ploso vi   
195                                             /kawan   
197                                                  /   
204                                      /dr man besar   
211                    ayam gepuk pak blangkon/dr. sut   
219                  masjid al-muttaqin/dukuh rejosari   
234  paguyuban pecinta batik indonesia sekar jagad/...   
237                                        /tenun suka   
250                           optical dewa/dep pay neg   
259  sekolah menengah pertama neg 19 samarinda/poro...   
262                                         /teru kopo   
266                                        /bante baru   
286              singapore station/jl.brigjend katamso   
294                                             /alun-   
295                                       /mojor gg. 8   

                                             predict5b  \
id                                                       
13                               warung bakso latansa/   
22                                            /gg. pan   
40                                 ruko cikitsu/jl abu   
44   komplek karya baru permai/jl. karya b

In [92]:
val_typo_err = val[(val['predict5'] != val['POI/street']) & (val['new_pred_list'].apply(lambda x: 'x'.join(x)) == val['y_list'].apply(lambda x: 'x'.join(x)))]

In [93]:
maping_street['par']

{'par': 688,
 'parman': 23,
 'parahy': 3,
 'parang': 9,
 'partow': 2,
 'parala': 1,
 'parigi': 3,
 'parung': 4,
 'paring': 1,
 'parkit': 4,
 'pariwi': 10,
 '.': 1}

In [94]:
val_typo_err[['raw_address', 'POI/street', 'predict5']]

raw_address  \
id                                                          
200040  kebayoran lama selatan bela apartemen icon, ke...   
107149  s par, no 51 santika department store & furnit...   
49370                     jend sudi no 35 mush al-rahman,   
94174                              wij vii kebayoran baru   
81271                      mi swasta al-f, sale ten baru,   
...                                                   ...   
294843  kli umum dok salik dan dok anggi, raya tonj ke...   
101059                          tahu susu, raya cin, no 9   
106076                           bal, petojo utara gambir   
297924              pondok man, yos suda singkawang barat   
165865                 beta rumah, said peri, no 53 97127   

                                               POI/street  \
id                                                          
200040               belakang apartemen icon/kebon man iv   
107149      santika department store & furniture/s parman   
49370                        musholla al-rahman/jend sudi   
94174                                         /wijaya vii   
81271                    mi swasta al-falah/sale ten baru   
...                                                   ...   
294843  klinik umum dokter salik dan dokter anggi/raya...   
101059                              tahu susu/raya cinere   
106076                                            /balikp   
297924                             pondok mangga/yos suda   
165865                           beta rumah/said perintah   

                                                 predict5  
id                                                         
200040                   bela apartemen icon/kebon man iv  
107149             santika depart store & furniture/s par  
49370                      musholla al - rahman/jend sudi  
94174                                            /wij vii  
81271                  mi swasta al - falah/sale ten baru  
...                                                   ...  
294843  klinik umum dokter salik dan dokter anggi/raya...  
101059                                 tahu susu/raya cin  
106076                                               /bal  
297924                                pondok man/yos suda  
165865                               beta rumah/said peri  

[7742 rows x 3 columns]

In [95]:
test['pred_poi'] = test['predict5'].str.split('/', n = 2, expand = True)[0]
test['pred_street'] = test['predict5'].str.split('/', n = 2, expand = True)[1]

In [96]:
test['poi_check'] = test[['raw_address','pred_poi']].apply(lambda x: x[1] in x[0], axis=1)
test['treet_check'] = test[['raw_address','pred_poi']].apply(lambda x: x[1] in x[0], axis=1)

In [97]:
test[test['raw_address'].apply(lambda x: 'bimbi' in x)]

raw_address  \
id                                                         
30          lemb bimbi belajar gane opera ahmad yani 19,   
3171                   kuta bimbi sun rd sun road, 80361   
6776      lemb bimbingan bela prim, raya kuta, kuta kuta   
14124  bimbingan bela rumah pin raya asto mulyo-punggur,   
22570  bimbi belajar duta pres, kakap, kuningan semar...   
37566          bimbingan prima siswa m, perm, tegal alur   

                                            clean_tokens  \
id                                                         
30     [lemb, bimbi, belajar, gane, opera, ahmad, yan...   
3171         [kuta, bimbi, sun, rd, sun, road, ,, 80361]   
6776   [lemb, bimbingan, bela, prim, ,, raya, kuta, ,...   
14124  [bimbingan, bela, rumah, pin, raya, asto, muly...   
22570  [bimbi, belajar, duta, pres, ,, kakap, ,, kuni...   
37566  [bimbingan, prima, siswa, m, ,, perm, ,, tegal...   

                                                  tokens  \
id                                                         
30     [lemb, bimbi, belajar, gane, opera, ahmad, yan...   
3171         [kuta, bimbi, sun, rd, sun, road, ,, 80361]   
6776   [lemb, bimbingan, bela, prim, ,, raya, kuta, ,...   
14124  [bimbingan, bela, rumah, pin, raya, asto, muly...   
22570  [bimbi, belajar, duta, pres, ,, kakap, ,, kuni...   
37566  [bimbingan, prima, siswa, m, ,, perm, ,, tegal...   

                                               pred_list  \
id                                                         
30     [B-POI, I-POI, I-POI, I-POI, I-POI, B-STREET, ...   
3171   [O, B-POI, I-POI, B-STREET, I-STREET, I-STREET...   
6776   [B-POI, I-POI, I-POI, I-POI, O, B-STREET, I-ST...   
14124  [B-POI, I-POI, I-POI, I-POI, B-STREET, I-STREE...   
22570              [O, O, O, O, O, B-STREET, O, O, O, O]   
37566  [B-POI, I-POI, I-POI, I-POI, O, B-STREET, O, O...   

                                               marginals  \
id                                                         
30     [{'O': 0.0005710618022779608, 'B-POI': 0.99922...   
3171   [{'O': 0.9999854160782612, 'B-POI': 8.45611732...   
6776   [{'O': 2.222752759497524e-06, 'B-POI': 0.99996...   
14124  [{'O': 0.0003132891258054746, 'B-POI': 0.99956...   
22570  [{'O': 0.7286927443598133, 'B-POI': 0.27122788...   
37566  [{'O': 0.0013990282280090735, 'B-POI': 0.99856...   

                                                    sent  \
id                                                         
30     [{'tag-argmax': 'v3', 'bert-argmax': 'v12', 'b...   
3171   [{'tag-argmax': 'v0', 'bert-argmax': 'v7', 'bi...   
6776   [{'tag-argmax': 'v3', 'bert-argmax': 'v38', 'b...   
14124  [{'tag-argmax': 'v3', 'bert-argmax': 'v11', 'b...   
22570  [{'tag-argmax': 'v3', 'bert-argmax': 'v12', 'b...   
37566  [{'tag-argmax': 'v3', 'bert-argmax': 'v11', 'b...   

                                                predict4  \
id                                                         
30     lembaga bimbi bela ganesha operation/ahmad yan...   
3171                               bimbi sun/rd sun road   
6776                lembaga bimbi bela primary/raya kuta   
14124    bimbi bela rumah pinang/raya asto mulyo-punggur   
22570                                             /kakap   
37566                           bimbi prima siswa m/perm   

                                              POI/street  \
id                                                         
30     lembaga bimbi bela ganesha operation/ahmad yan...   
3171                               bimbi sun/rd sun road   
6776                lembaga bimbi bela primary/raya kuta   
14124    bimbi bela rumah pinang/raya asto mulyo-punggur   
22570                     bimbi bela duta prestasi/kakap   
37566                           bimbi prima siswa m/perm   

                                          pred_marginals  \
id                                                         
30     [B-POI, I-POI, I-POI, I-POI, I-POI, B-STREET, ...   
3171   [O, B-POI,

In [98]:
val[val['raw_address'].apply(lambda x: 'bimbi' in x)][['raw_address', 'predict5', 'POI/street']].head(5)

raw_address  \
id                                                          
93049                bimbi bela cec, sri rej, no 33 63114   
232378  brig ngu rai no 36 lemb bimbi belajar prima - ...   
195227                bimbi belajar kris, pelab pekutatan   
151684  bimbingan dan konsultasi bela adz, al bai, 40 ...   
69871        lemb bimbingan belajar gamma jogja, jaya wij   

                                                 predict5  \
id                                                          
93049                              bimbi bela cec/sri rej   
232378  lembaga bimbi bela prima - bangli bali/brig ng...   
195227                                   bimbi bela kris/   
151684                                            /al bai   
69871             lembaga bimbi bela gamma jogja/jaya wij   

                                               POI/street  
id                                                         
93049                       bimbingan belajar cec/sri rej  
232378  lembaga bimbingan belajar primagama - bangli b...  
195227                    bimbingan belajar krishna/pelab  
151684                                            /al bai  
69871      lembaga bimbingan belajar gamma jogja/jaya wij

In [99]:
test[~test['poi_check']][['raw_address', 'pred_poi']].head(20)

raw_address  \
id                                                       
2                                asma laun, mand imog,   
5    pem dos dapur ala perum gar no a 12 suka jaya ...   
6                                   tb. mara, cisayong   
13                        war bakso lata kota waingapu   
16          dimas laun demak, jl tanjung ii, katonsari   
30        lemb bimbi belajar gane opera ahmad yani 19,   
36   m. t. hary, no 11 bank neg indonesia kali rejo...   
48                        cv pon rami 23373 darussalam   
52                         mas nurul huda badang ngoro   
62          lang waqof hidayatul ummah mast warugunung   
63   tanjung mulia hilir s.j lap service, kawat i no 2   
85   pem palembang musi raya matr raya, pal meriem ...   
88        h imam muna, no 65 crys computer, bukit raya   
90   koperasi sim pinjam sinar pel, pelab bojong ci...   
107    move on cell, pasundan no 1 75123 samarinda ulu   
113   sd negeri 3 seunuddon 24393 seunuddon (seunudon)   
117      ade cell maos raya maos kidul, klapagada maos   
123     terapi mir plus 8 als box, beng, bandung wetan   
131                  califo fried chic dip, proklamasi   
133                     cv. sinar mand tek soek hatta,   

                                 pred_poi  
id                                         
2                            asma laundry  
5                    pempek dos dapur ala  
6                            tb. maranggi  
13                   warung bakso latansa  
16                    dimas laundry demak  
30   lembaga bimbi bela ganesha operation  
36                  bank negara indonesia  
48                      cv pondok raminih  
52                      masjid nurul huda  
62         langgeng waqof hidayatul ummah  
63                    s. j laptop service  
85                  pempek palembang musi  
88                       crystal computer  
90    koperasi simpan pinjam sinar pelita  
107                      movement on cell  
113                    sd neg 3 seunuddon  
117                      ade cell maoslor  
123          terapi miroso plus 8 als box  
131              california fried chicken  
133              cv. sinar mandiri teknik